In [128]:
import requests,os,time
def ratelimit():
    "A function that handles the rate of your calls."
    time.sleep(0.5) # sleep one second.

class Connector():
  def __init__(self,logfile,overwrite_log=False,connector_type='requests',session=False,path2selenium='',n_tries = 5,timeout=30):
    """This Class implements a method for reliable connection to the internet and monitoring. 
    It handles simple errors due to connection problems, and logs a range of information for basic quality assessments
    
    Keyword arguments:
    logfile -- path to the logfile
    overwrite_log -- bool, defining if logfile should be cleared (rarely the case). 
    connector_type -- use the 'requests' module or the 'selenium'. Will have different since the selenium webdriver does not have a similar response object when using the get method, and monitoring the behavior cannot be automated in the same way.
    session -- requests.session object. For defining custom headers and proxies.
    path2selenium -- str, sets the path to the geckodriver needed when using selenium.
    n_tries -- int, defines the number of retries the *get* method will try to avoid random connection errors.
    timeout -- int, seconds the get request will wait for the server to respond, again to avoid connection errors.
    """
    
    ## Initialization function defining parameters. 
    self.n_tries = n_tries # For avoiding triviel error e.g. connection errors, this defines how many times it will retry.
    self.timeout = timeout # Defining the maximum time to wait for a server to response.
    ## not implemented here, if you use selenium.
    if connector_type=='selenium':
      assert path2selenium!='', "You need to specify the path to you geckodriver if you want to use Selenium"
      from selenium import webdriver 
      ## HIN download the latest geckodriver here: https://github.com/mozilla/geckodriver/releases

      assert os.path.isfile(path2selenium),'You need to insert a valid path2selenium the path to your geckodriver. You can download the latest geckodriver here: https://github.com/mozilla/geckodriver/releases'
      self.browser = webdriver.Firefox(executable_path=path2selenium) # start the browser with a path to the geckodriver.

    self.connector_type = connector_type # set the connector_type
    
    if session: # set the custom session
      self.session = session
    else:
      self.session = requests.session()
    self.logfilename = logfile # set the logfile path
    ## define header for the logfile
    header = ['id','project','connector_type','t', 'delta_t', 'url', 'redirect_url','response_size', 'response_code','success','error']
    if os.path.isfile(logfile):        
      if overwrite_log==True:
        self.log = open(logfile,'w')
        self.log.write(';'.join(header))
      else:
        self.log = open(logfile,'a')
    else:
      self.log = open(logfile,'w')
      self.log.write(';'.join(header))
    ## load log 
    with open(logfile,'r') as f: # open file
        
      l = f.read().split('\n') # read and split file by newlines.
      ## set id
      if len(l)<=1:
        self.id = 0
      else:
        self.id = int(l[-1][0])+1
            
  def get(self,url,project_name):
    """Method for connector reliably to the internet, with multiple tries and simple error handling, as well as default logging function.
    Input url and the project name for the log (i.e. is it part of mapping the domain, or is it the part of the final stage in the data collection).
    
    Keyword arguments:
    url -- str, url
    project_name -- str, Name used for analyzing the log. Use case could be the 'Mapping of domain','Meta_data_collection','main data collection'. 
    """
     
    project_name = project_name.replace(';','-') # make sure the default csv seperator is not in the project_name.
    if self.connector_type=='requests': # Determine connector method.
      for _ in range(self.n_tries): # for loop defining number of retries with the requests method.
        ratelimit()
        t = time.time()
        try: # error handling 
          response = self.session.get(url,timeout = self.timeout) # make get call

          err = '' # define python error variable as empty assumming success.
          success = True # define success variable
          redirect_url = response.url # log current url, after potential redirects 
          dt = t - time.time() # define delta-time waiting for the server and downloading content.
          size = len(response.text) # define variable for size of html content of the response.
          response_code = response.status_code # log status code.
          ## log...
          call_id = self.id # get current unique identifier for the call
          self.id+=1 # increment call id
          #['id','project_name','connector_type','t', 'delta_t', 'url', 'redirect_url','response_size', 'response_code','success','error']
          row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row to be written in the log.
          self.log.write('\n'+';'.join(map(str,row))) # write log.
          self.log.flush()
          return response,call_id # return response and unique identifier.

        except Exception as e: # define error condition
          err = str(e) # python error
          response_code = '' # blank response code 
          success = False # call success = False
          size = 0 # content is empty.
          redirect_url = '' # redirect url empty 
          dt = t - time.time() # define delta t

          ## log...
          call_id = self.id # define unique identifier
          self.id+=1 # increment call_id

          row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row
          self.log.write('\n'+';'.join(map(str,row))) # write row to log.
          self.log.flush()
    else:
      t = time.time()
      ratelimit()
      self.browser.get(url) # use selenium get method
      ## log
      call_id = self.id # define unique identifier for the call. 
      self.id+=1 # increment the call_id
      err = '' # blank error message
      success = '' # success blank
      redirect_url = self.browser.current_url # redirect url.
      dt = t - time.time() # get time for get method ... NOTE: not necessarily the complete load time.
      size = len(self.browser.page_source) # get size of content ... NOTE: not necessarily correct, since selenium works in the background, and could still be loading.
      response_code = '' # empty response code.
      row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row 
      self.log.write('\n'+';'.join(map(str,row))) # write row to log file.
      self.log.flush()
    # Using selenium it will not return a response object, instead you should call the browser object of the connector.
    ## connector.browser.page_source will give you the html.
      return None,call_id

In [243]:

import bs4 as bs
from urllib.request import urlopen as ureq
from bs4 import BeautifulSoup as soup
import pandas as pd
import urllib.request
import re


url = "https://www.tripadvisor.com//Restaurant_Review-g189541-d16818421-Reviews-Mulgeo_Blinis_Bubbles_Bar_Torvehallerne-Copenhagen_Zealand.html"

In [244]:
#connector = Connector('logfile_exam_test.csv')

name = url.replace('https://www.tripadvisor.com//Restaurant_Review-', '')
name = name.replace('-Reviews-', '')
name = name.replace('-Copenhagen_Zealand.html', '')
name = name.replace('_', ' ')
name = name[17:]

#url = "https://www.tripadvisor.com/RestaurantSearch-g189541-o120-a_date.2019__2D__08__2D__22-a_people.2-a_time.20%3A00%3A00-a_zur.2019__5F__08__5F__22-Copenhag.html#EATERY_LIST_CONTENTS"
trip = ureq(url)
trip_html = trip.read()
trip.close()

trip_soup = soup(trip_html, "lxml")
#finds_food = str(trip_soup.find_all("span",class_="restaurants-detail-overview-cards-RatingsOverviewCard__ratingText--1P1Lq"))
#finds_c = str(trip_soup.find_all("span",class_="restaurants-detail-overview-cards-RatingsOverviewCard__ratingBubbles--1kQYC"))
test = trip_soup.findAll(True, {"class":["restaurants-detail-overview-cards-RatingsOverviewCard__ratingText--1P1Lq", "restaurants-detail-overview-cards-RatingsOverviewCard__ratingBubbles--1kQYC"]})
#response,call_id = connector.get(url,'course_mapping')
#locations = response.text.split('<div>')
test

elements = []
for x in test:
    elements.append(str(x))
    

print(name)

Mulgeo Blinis Bubbles Bar Torvehallerne


In [245]:
type(elements)
keys = elements[0::2]
values = elements[1::2]
keys.append("Name")
values.append(str(name))

keys



['Name']

In [246]:
keys[:] = [s.replace('<span class="restaurants-detail-overview-cards-RatingsOverviewCard__ratingText--1P1Lq">', '') for s in keys]
keys[:] = [s.replace('</span>', '') for s in keys]
values[:] = [s.replace('<span class="restaurants-detail-overview-cards-RatingsOverviewCard__ratingBubbles--1kQYC"><span class="ui_bubble_rating bubble_', '') for s in values]
values[:] = [s.replace('"></span></span>', '') for s in values]

keys
values


['Mulgeo Blinis Bubbles Bar Torvehallerne']

In [247]:
ratings_dict = {}
for i in range(len(keys)):
    ratings_dict[keys[i]] = values[i]
    
ratings_df = ratings_df.append([ratings_dict], ignore_index=True)

C:\Users\sofie\Anaconda3\lib\site-packages\pandas\core\indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [248]:
ratings_df.head()

,God pris,Mad,Name,Service,Stemning
0,45,45,The Red Box,50,45
1,45,45,The Red Box,50,45
2,NaN,NaN,Mulgeo Blinis Bubbles Bar Torvehallerne,NaN,NaN


In [301]:
# samlet kode
# import urls: 
urls = pd.read_csv("page_urls.csv")

In [302]:
df_urls = pd.DataFrame(urls)
ratings_df = pd.DataFrame()

In [ ]:
ratings_df = pd.DataFrame()
count = 0
for url in df_urls["0"]:
    count = count+1
    print(count)

    trip = ureq(url)
    trip_html = trip.read()
    trip.close()

    trip_soup = soup(trip_html, "lxml")
    test = trip_soup.findAll(True, {"class":["restaurants-detail-overview-cards-RatingsOverviewCard__ratingText--1P1Lq", "restaurants-detail-overview-cards-RatingsOverviewCard__ratingBubbles--1kQYC"]})
    name = trip_soup.findAll(True, {"class":["ui_header h1"]})
    name = str(name)
    name = name[26:]
    name = name.replace("</h1>]", '')
      
    elements = []
    for x in test:
        elements.append(str(x))
    
    keys = elements[0::2]
    values = elements[1::2]
    keys.append("Name")
    values.append(str(name))

    keys[:] = [s.replace('<span class="restaurants-detail-overview-cards-RatingsOverviewCard__ratingText--1P1Lq">', '') for s in keys]
    keys[:] = [s.replace('</span>', '') for s in keys]
    values[:] = [s.replace('<span class="restaurants-detail-overview-cards-RatingsOverviewCard__ratingBubbles--1kQYC"><span class="ui_bubble_rating bubble_', '') for s in values]
    values[:] = [s.replace('"></span></span>', '') for s in values]
    
    ratings_dict = {}
    for i in range(len(keys)):
        ratings_dict[keys[i]] = values[i]
    
    ratings_df = ratings_df.append([ratings_dict], ignore_index=True)
    

1
2
3


C:\Users\sofie\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


4
5


C:\Users\sofie\Anaconda3\lib\site-packages\pandas\core\indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
28

In [300]:
df.to_csv

,Atmosphere,Food,Name,Service,Value
0,NaN,50,The Pescatarian,50,45
1,NaN,45,Trattoria SUD,50,45
2,45,50,Enomania,50,45
3,45,50,Geranium,45,45
4,NaN,50,Garden Restaurant &amp; Bar,45,45
5,45,45,Frederiks Have,45,45
6,45,50,Studio - The Standard,50,45
7,NaN,50,The Olive Kitchen &amp; Bar,50,45
8,NaN,50,Cafe Komplet,50,45
9,NaN,45,Basso Kobenhavn,45,45
